## Linear Model for MNIST dataset

We are going to do a linear model for the MNIST Dataset.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
tf.summary.FileWriterCache.clear()

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
if IN_COLAB == True:
    !pip install -U tensorboardcolab
    from tensorboardcolab import *
    tbc=TensorBoardColab()


## Step 1: Load the MNIST data

In [ ]:
#Load Mnist data

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


Now let's take a look at one of these downloaded images:

In [ ]:
plt.imshow(mnist.train.images[1].reshape([28,28]))

#TODO: Try plotting another image



## Step 2: Define our model and equation.

Our formula is:

$$y = Wx + B$$

Straight from your algebra course.  Of course, in this case our input is a 28 by 28 matrix, which we will flatten in into a vector of 784.  

Each variable is going to be one-hot-encoded, to be a one-hot array of 10. Why 10? 10 possible values in the classification.

So our weight is going to be a 784 x 10 matrix, and the bias will be a vector of size 10.

The placeholder will have a initial param of None so we can feed all the training samples.

In [ ]:
#Define Variable placeholders

x = tf.placeholder(tf.float32, [None, 784], name='Input')
W = tf.Variable(tf.zeros([784, 10]), name='Weight')
b = tf.Variable(tf.zeros([10]), name='Bias')
logs_path = '/tmp/tensorflow_logs/example/'

In [ ]:
#Set up softmax regression model
with tf.name_scope('model'):
    y = tf.nn.softmax(tf.matmul(x, W) + b)


## Step 3: Train the Model

We will define our y_ which is the placeholder for our labels.

We will also define our cross-entropy loss which is:

$$H(p,q) = -\sum_{x\in\mathcal{X}} p(x)\, \log q(x)$$


**=> TODO: Set learning Rate to 0.5**

In [ ]:
# Train model
y_ = tf.placeholder(tf.float32, [None, 10])

with tf.name_scope('Correctness'):
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    
with tf.name_scope('Accuracy'):
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Add summaries to collect information needed for tensorboard
tf.summary.histogram("Weight", W)
tf.summary.histogram("Bias", b)
tf.summary.scalar("acc", accuracy)
merge=tf.summary.merge_all()

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

with tf.name_scope('SGD'):
    train_step = tf.train.GradientDescentOptimizer(???).minimize(cross_entropy)
    
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# Write summaries into the file

summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph()) #write the logs 


**=> TODO: Set the Number of epochs to 1000 **


In [ ]:

for _ in range(???):
  batch_xs, batch_ys = mnist.train.next_batch(100)


  nothing, summary = sess.run([train_step,merge], feed_dict={x: batch_xs, y_: batch_ys})
  
  summary_writer.add_summary(summary, _ )

try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
if IN_COLAB == True:
    summary_writer = tbc.get_writer()# saving data on google colab
    summary_writer.add_graph(sess.graph)
  


## Step 4: Get our Accuracy

The accuracy is the number correct over the total. Let's get that.

In [ ]:
# Evaluate Model


print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))


## Step 5: Look at our Graph

In [ ]:
## Step 5: Look at our Graph
# Run this "helper" code to visualize the session graph.

from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "b<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [ ]:
show_graph(sess.graph)


### Now we run the TensorBaord
#### Local:

Now run the following command on a bash

```bash
$ tensorboard --logdir=/tmp/tensorflow_logs
```
Then open the browser(Chrome) and insert this address

```bash
localhost:6006
```

#### Google colab



Click on the link you had got at the first step

![](../images/tb_colab.png)